<a href="https://colab.research.google.com/github/FlorianBoecker/teaching/blob/master/AppliedBioinformatics/Notebooks/prepare_AgriGO_SEA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prepare Gene Level Expression Data for PAGE analysis in AgriGO

##Install and load necessary R packages

In [ ]:
R.Version()

install.packages("statmod", verbose = TRUE)
install.packages("BiocManager", verbose = TRUE)
BiocManager::install(ask = FALSE)
BiocManager::install("limma")
BiocManager::install("edgeR")

In [ ]:
library(statmod)
library(limma)
library(edgeR)

##Load the data

Under the github link we provided in the exercise sheets (https://github.com/tgstoecker/teaching/tree/master/AppliedBioinformatics/Results_2021), you can choose the featureCounts output you or your group generated.  
Just exchange its raw data link for the one shown here. 

In [ ]:
counts <- "https://raw.githubusercontent.com/tgstoecker/teaching/master/AppliedBioinformatics/B73/gene-level/total_file.count"
fc_res <- read.table(counts, header = T, row.names = 1)

Shorten the column names indicating the samples - e.g.:


In [ ]:
colnames(fc_res) <- sub("_trimmed_sorted.bam", "", colnames(fc_res))

## Determine differentially expressed genes

Create a vector indicating treatment conditions of the samples 
- logic: columns left to right

In [ ]:
group = c("control", "control", "control", "control", "drought", "drought", "drought", "drought")

Create a DGE list object

In [ ]:
dge = DGEList(counts = fc_res[, 6:13], group = group, genes = rownames(fc_res))

Transformation of raw read counts

In [ ]:
cpm <- cpm(dge)
lcpm <- cpm(dge, log=TRUE)

Create design matrix

In [ ]:
design <- model.matrix(~0+group)

Filter

In [ ]:
keep <- filterByExpr(dge, design)
dge_filtered <- dge[keep, , keep.lib.sizes=FALSE]

Normalization

In [ ]:
dge_normalized <- calcNormFactors(dge_filtered, method = "TMM")

Dispersion estimation

In [ ]:
dge_disp <- estimateDisp(dge_normalized, design, robust=TRUE)
fit <- glmQLFit(dge_disp, design, robust=TRUE)

Test for differential expression

In [ ]:
CvsD <- makeContrasts(groupdrought-groupcontrol, levels=design)
res <- glmQLFTest(fit, contrast=CvsD)

Select up and down regulated genes with a logFC and an FDR treshold

In [ ]:
check <- topTags(res, adjust.method = "BH", n = "all")
# If there is time, you can experiment with different values for the thesholds!
up_drought <- subset(check$table, logFC > 1 & FDR < 0.05)$genes
down_drought <- subset(check$table, logFC < -1 & FDR < 0.05)$genes
length(up_drought)
length(down_drought)

## Write data to file

In [ ]:
# Don't forget to change the filename to something that is appropriate to your genotype!
write.table(up_drought, "b73_upRegulatedGenes.tsv", quote=F, row.names=F, col.names=F)
write.table(down_drought, "b73_downRegulatedGenes.tsv", quote=F, row.names=F, col.names=F)